## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,0,Luderitz,-26.6481,15.1594,55.29,75,0,15.12,NaN,clear sky,2022-10-04 22:53:29
1,1,Tomatlan,19.9333,-105.2500,81.55,78,55,6.20,MX,broken clouds,2022-10-04 22:53:30
2,2,Clyde River,70.4692,-68.5914,26.83,86,100,17.27,CA,overcast clouds,2022-10-04 22:53:30
3,3,Coahuayana,18.7333,-103.6833,82.67,77,69,3.98,MX,broken clouds,2022-10-04 22:53:31
4,4,Hamilton,39.1834,-84.5333,71.04,39,0,9.22,US,clear sky,2022-10-04 22:49:04


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("How cold of a trip do you want? "))
max_temp = float(input("How hot of a trip do you want? "))

How cold of a trip do you want? 75
How hot of a trip do you want? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
1,1,Tomatlan,19.9333,-105.2500,81.55,78,55,6.20,MX,broken clouds,2022-10-04 22:53:30
3,3,Coahuayana,18.7333,-103.6833,82.67,77,69,3.98,MX,broken clouds,2022-10-04 22:53:31
8,8,Butaritari,3.0707,172.7902,80.89,79,13,15.08,KI,light rain,2022-10-04 22:53:33
16,16,Vaini,-21.2000,-175.2000,82.56,78,75,5.75,TO,broken clouds,2022-10-04 22:53:36
22,22,Avarua,-21.2078,-159.7750,78.85,69,20,19.57,CK,few clouds,2022-10-04 22:53:38
27,27,Kalmunai,7.4167,81.8167,80.40,73,100,3.56,LK,overcast clouds,2022-10-04 22:53:40
28,28,Majene,-3.5403,118.9707,78.53,85,100,3.76,ID,overcast clouds,2022-10-04 22:53:40
29,29,Victoria,22.2855,114.1577,86.18,86,55,8.01,HK,broken clouds,2022-10-04 22:52:15
33,33,Bathsheba,13.2167,-59.5167,81.21,89,20,24.16,BB,few clouds,2022-10-04 22:53:41
38,38,Acajutla,13.5928,-89.8275,81.77,79,91,6.38,SV,overcast clouds,2022-10-04 22:53:43


In [7]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.isnull().sum()

217


City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Weather Description    0
Date                   0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_preferred_cities_df))
clean_preferred_cities_df.head()

217


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
1,1,Tomatlan,19.9333,-105.2500,81.55,78,55,6.20,MX,broken clouds,2022-10-04 22:53:30
3,3,Coahuayana,18.7333,-103.6833,82.67,77,69,3.98,MX,broken clouds,2022-10-04 22:53:31
8,8,Butaritari,3.0707,172.7902,80.89,79,13,15.08,KI,light rain,2022-10-04 22:53:33
16,16,Vaini,-21.2000,-175.2000,82.56,78,75,5.75,TO,broken clouds,2022-10-04 22:53:36
22,22,Avarua,-21.2078,-159.7750,78.85,69,20,19.57,CK,few clouds,2022-10-04 22:53:38


In [12]:
clean_preferred_cities_df.columns =  ['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Current Description', 'Date']

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tomatlan,MX,81.55,broken clouds,19.9333,-105.2500,
3,Coahuayana,MX,82.67,broken clouds,18.7333,-103.6833,
8,Butaritari,KI,80.89,light rain,3.0707,172.7902,
16,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
22,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
27,Kalmunai,LK,80.40,overcast clouds,7.4167,81.8167,
28,Majene,ID,78.53,overcast clouds,-3.5403,118.9707,
29,Victoria,HK,86.18,broken clouds,22.2855,114.1577,
33,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
38,Acajutla,SV,81.77,overcast clouds,13.5928,-89.8275,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tomatlan,MX,81.55,broken clouds,19.9333,-105.2500,El Salvial
3,Coahuayana,MX,82.67,broken clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
8,Butaritari,KI,80.89,light rain,3.0707,172.7902,Isles Sunset Lodge
16,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
682,Sakassou,CI,78.64,light rain,7.4546,-5.2926,Parents de Damienne
684,Hope,US,82.08,clear sky,33.6671,-93.5916,Best Western Hope
687,Amahai,ID,80.08,overcast clouds,-3.3333,128.9167,HOTEL ISABELA
694,Dabhol,IN,76.01,overcast clouds,17.6000,73.1667,Ashwamedh Resort


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))